Let's unfuck this shall we.

In [71]:
from umatrix import *

class Kinematics:
    WHEEL_RADIUS = 0.030
    TRACK_WIDTH  = 0.188 
    TRACK_LENGTH = 0.147

    l = TRACK_LENGTH / 2
    w = TRACK_WIDTH  / 2
    r = WHEEL_RADIUS

    # The kinematic model assumes that the motors are laid out as follows:
    #  [3]     [1]
    #     ---> x
    #  [4]     [2]

    k_const = (l+w)
    kinematic_model = matrix(
        [ 1,  1, -k_const], 
        [ 1, -1,  k_const], 
        [ 1, -1, -k_const], 
        [ 1,  1,  k_const]) * (1/r)
    
    inverse_kinematic_model = matrix(
        [          1,           1,             1,           1],
        [          1,          -1,            -1,           1],
        [-(1/k_const), (1/k_const),  -(1/k_const), (1/k_const)]) * (r/4)


    def motor_speeds(self, vx, vy, wz):
        input_array = matrix([vx],[vy],[wz])    
        return self.kinematic_model * input_array


    def velocities(self, speeds):     
        return self.inverse_kinematic_model * speeds
        
        
    def odometry(self, velocities, duration):    
        u = self.inverse_kinematic_model * velocities    
        return u * duration   
        

In [85]:
import time 

kn = Kinematics()

duration = 0.000027

speed_time = time.time_ns()

speeds = kn.motor_speeds(1, 0, 0)
speed_time = time.time_ns() - speed_time
print(f"speeds: {speeds}, time: {speed_time/ 1000000000}ms")

# print(speeds[0])
velo_times = time.time_ns()
velos = kn.velocities(speeds)
velo_times = time.time_ns() - velo_times
print(f"velos: {velos}, time: {velo_times/ 1000000000}ms")


one_ms = 1 / 1000
one_us = 1 / 1000000

motor_speeds = matrix([8.181559], [8.63254], [8.123714], [8.055882])
odom_time = time.time_ns()
# odom = kn.odometry(motor_speeds, 0.000027)
odom = kn.odometry(speeds, 0.000027)
odom_time = time.time_ns() - odom_time
print(f"odom: {odom}, time: {odom_time / 1000000000}ms")

speeds: [33.333333333333336,
 33.333333333333336,
 33.333333333333336,
 33.333333333333336], time: 0.000166482ms
velos: [1.0,
 0.0,
 0.0], time: 0.000133371ms
odom: [2.7e-05,
     0.0,
     0.0], time: 0.000147167ms
